# Analyzing Thanksgiving Dinner
In this project, you'll be working with Jupyter notebook, and analyzing data on Thanksgiving dinner in the US.

The dataset is stored in the thanksgiving-2015-poll-data.csv file. It contains 1058 responses to an online survey about what Americans eat for Thanksgiving dinner. Each survey respondent was asked questions about what they typically eat for Thanksgiving, along with some demographic questions, like their gender, income, and location. This dataset will allow us to discover regional and income-based patterns in what Americans eat for Thanksgiving dinner.

The dataset has 65 columns, and 1058 rows. Most of the column names are questions, and most of the column values are string responses to the questions. Most of the columns are categorical, as a survey respondent had to select one of a few options. For example, one of the first column names is What is typically the main dish at your Thanksgiving dinner?. The potential responses are:
- Turkey
- Other (please specify)
- Ham/Pork
- Tofurkey
- Chicken
- Roast beef
- I don't know
- Turducken

Most of the columns follow the same question/response format as the above. There are also quite a few NaN values in the columns, which occurred when a survey respondent didn't fill out a question because they didn't want to, or it didn't apply to them.


Here are the first few rows of the dataset:

In [1]:
import pandas as pd

data = pd.read_csv("thanksgiving-2015-poll-data.csv",encoding="Latin-1")

print data.head(5)

   RespondentID Do you celebrate Thanksgiving?  \
0    4337954960                            Yes   
1    4337951949                            Yes   
2    4337935621                            Yes   
3    4337933040                            Yes   
4    4337931983                            Yes   

  What is typically the main dish at your Thanksgiving dinner?  \
0                                             Turkey             
1                                             Turkey             
2                                             Turkey             
3                                             Turkey             
4                                           Tofurkey             

  What is typically the main dish at your Thanksgiving dinner? - Other (please specify)  \
0                                                NaN                                      
1                                                NaN                                      
2                            

We won't enumerate every single column now, but here are descriptions of some of the most important:
- RespondentID -- a unique ID of the respondent to the survey.
- Do you celebrate Thanksgiving? -- a Yes/No reponse to the question.
- How would you describe where you live? -- responses are Suburban, Urban, and Rural.
- Age -- resposes are one of several categories, such as 18-29, and 30-44.
- How much total combined money did all members of your HOUSEHOLD earn last year? -- one of several categories, such as $75,000 to $99,999.

In this project, we'll explore the data, and try to find interesting patterns. Our first step is to read in and display the data.

Let's take a look to the columns of the dataset:

In [2]:
print(data.columns)

Index([u'RespondentID', u'Do you celebrate Thanksgiving?',
       u'What is typically the main dish at your Thanksgiving dinner?',
       u'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       u'How is the main dish typically cooked?',
       u'How is the main dish typically cooked? - Other (please specify)',
       u'What kind of stuffing/dressing do you typically have?',
       u'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       u'What type of cranberry saucedo you typically have?',
       u'What type of cranberry saucedo you typically have? - Other (please specify)',
       u'Do you typically have gravy?',
       u'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       u'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       u'Which of these side dishes arety

Because we want to understand what people ate for Thanksgiving, we'll remove any responses from people who don't celebrate it. The column Do you celebrate Thanksgiving? contains this information. We only want to keep data for people who answered Yes to this questions.

In [3]:
print(data["Do you celebrate Thanksgiving?"].value_counts())

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64


In [4]:
filtered_data = data[data["Do you celebrate Thanksgiving?"] == "Yes"]
print(filtered_data["Do you celebrate Thanksgiving?"].value_counts())

Yes    980
Name: Do you celebrate Thanksgiving?, dtype: int64


Let's explore what main dishes people tend to eat during Thanksgiving dinner. We can again use the value_counts method to help us with this.

In [5]:
print(filtered_data["What is typically the main dish at your Thanksgiving dinner?"].value_counts())

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64


In [6]:
print(
    filtered_data[
        filtered_data["What is typically the main dish at your Thanksgiving dinner?"] == "Tofurkey"
    ]["Do you typically have gravy?"]
)

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object


Exploring the desserts:

In [11]:
apple_isnull =data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"].isnull()
pumpkin_isnull = data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"].isnull()
pecan_isnull = data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"].isnull()
no_pies = apple_isnull & pumpkin_isnull & pecan_isnull
print(no_pies.value_counts())

False    876
True     182
dtype: int64


Exploring the Age column

In [13]:
def parse_age(age):
    if(pd.isnull(age)):
        return None
    age_token = age.split(' ')[0]
    if age_token.endswith('+'):
        return int(age_token.split('+')[0])
    return int(age_token)

data["int_age"] = data["Age"].apply(parse_age)
print(data["int_age"].describe())

count    1025.000000
mean       39.383415
std        15.398493
min        18.000000
25%        30.000000
50%        45.000000
75%        60.000000
max        60.000000
Name: int_age, dtype: float64


We should be aware about the null values in int_age, also this type of assigning an integer value to that column could lead to false assumptions because we are converting a period into a concrete year, also we are ommitting the null values and giving the same importance to the 60+ value to the 18-29 value, and we have to be carefull with that, because in one the period can be 4 times bigger that the other one.

Now let's take a look to the HOUSEHOLD column:

In [17]:
def parse_household(household):
    if pd.isnull(household):
        return None
    household_token = household.split(' ')[0]
    if household_token == "Prefer":
        return None
    value = household_token.replace(',','')
    return int(value.replace('$',''))

data["int_income"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(parse_household)
print(data["int_income"].describe())

count       889.000000
mean      74077.615298
std       59360.742902
min           0.000000
25%       25000.000000
50%       50000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64


Here we have the same problems that we had with the age column, teh null values, the fact that we are treating with ranges, the values that we assume that are null, and the difference between the upper range and the lower range.

Now let's try to find a correlation between the income and the travel distance:


In [19]:
travel_less_150000 = data[data["int_income"] < 150000]["How far will you travel for Thanksgiving?"]
print(travel_less_150000.value_counts())

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64


In [20]:
travel_more_150000 = data[data["int_income"] >= 150000]["How far will you travel for Thanksgiving?"]
print(travel_more_150000.value_counts())

Thanksgiving is happening at my home--I won't travel at all                         66
Thanksgiving is local--it will take place in the town I live in                     34
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    25
Thanksgiving is out of town and far away--I have to drive several hours or fly      15
Name: How far will you travel for Thanksgiving?, dtype: int64


In the first case there are up to 5~6 times more people having the dinner in their homes than flying several hours.
In the second case this ratio lows until 4 times more or less.

Now let's explore the data related to friendship columns:


In [21]:
pivot_table = pd.pivot_table(data,index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",columns='Have you ever attended a "Friendsgiving?"',values="int_age")
print(pivot_table)

Have you ever attended a "Friendsgiving?"                  No        Yes
Have you ever tried to meet up with hometown fr...                      
No                                                  42.283702  37.010526
Yes                                                 41.475410  33.976744


In [29]:
pivot_table = pd.pivot_table(data,index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",columns='Have you ever attended a "Friendsgiving?"',values="int_income")
print(pivot_table)

Have you ever attended a "Friendsgiving?"                     No           Yes
Have you ever tried to meet up with hometown fr...                            
No                                                  78914.549654  72894.736842
Yes                                                 78750.000000  66019.736842
